# 03. 音響合成とシンセサイザー

プログラムによる音響合成技術を学び、シンプルなシンセサイザーを実装します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. 基本波形の生成

In [ ]:
class WaveformGenerator:
    def __init__(self, sample_rate=44100):
        self.sample_rate = sample_rate
        
    def sine_wave(self, frequency, duration, amplitude=1.0, phase=0):
        """サイン波"""
        t = np.linspace(0, duration, int(self.sample_rate * duration), False)
        return t, amplitude * np.sin(2 * np.pi * frequency * t + phase)
        
    def square_wave(self, frequency, duration, amplitude=1.0, duty_cycle=0.5):
        """方形波"""
        t = np.linspace(0, duration, int(self.sample_rate * duration), False)
        return t, amplitude * signal.square(2 * np.pi * frequency * t, duty=duty_cycle)
        
    def sawtooth_wave(self, frequency, duration, amplitude=1.0):
        """のこぎり波"""
        t = np.linspace(0, duration, int(self.sample_rate * duration), False)
        return t, amplitude * signal.sawtooth(2 * np.pi * frequency * t)
        
    def triangle_wave(self, frequency, duration, amplitude=1.0):
        """三角波"""
        t = np.linspace(0, duration, int(self.sample_rate * duration), False)
        return t, amplitude * signal.sawtooth(2 * np.pi * frequency * t, width=0.5)
        
    def noise(self, duration, amplitude=1.0, noise_type='white'):
        """ノイズ生成"""
        samples = int(self.sample_rate * duration)
        t = np.linspace(0, duration, samples, False)
        
        if noise_type == 'white':
            noise = np.random.normal(0, amplitude, samples)
        elif noise_type == 'pink':
            # 1/f ノイズの近似
            white = np.random.normal(0, 1, samples)
            freqs = fftfreq(samples, 1/self.sample_rate)
            fft_white = fft(white)
            # 周波数に反比例する重み
            weights = 1 / np.sqrt(np.abs(freqs) + 1)
            fft_pink = fft_white * weights
            noise = np.real(np.fft.ifft(fft_pink)) * amplitude
        
        return t, noise

# 基本波形の可視化
sample_rate = 8000
gen = WaveformGenerator(sample_rate)
frequency = 440  # A4
duration = 0.01  # 10ms（波形の詳細を見るため）

waveforms = {
    'サイン波': gen.sine_wave(frequency, duration),
    '方形波': gen.square_wave(frequency, duration),
    'のこぎり波': gen.sawtooth_wave(frequency, duration),
    '三角波': gen.triangle_wave(frequency, duration),
    'ホワイトノイズ': gen.noise(duration, 0.5, 'white'),
    'ピンクノイズ': gen.noise(duration, 0.5, 'pink')
}

plt.figure(figsize=(18, 12))

for i, (name, (t, wave)) in enumerate(waveforms.items()):
    # 時間波形
    plt.subplot(3, 4, i*2 + 1)
    plt.plot(t * 1000, wave, linewidth=2)  # ms単位
    plt.title(f'{name}（時間波形）')
    plt.xlabel('時間 (ms)')
    plt.ylabel('振幅')
    plt.grid(True)
    
    # スペクトル
    plt.subplot(3, 4, i*2 + 2)
    
    # より長い信号でスペクトル解析
    if 'ノイズ' not in name:
        t_long, wave_long = getattr(gen, name.replace('波', '_wave').replace('サイン', 'sine').replace('方形', 'square').replace('のこぎり', 'sawtooth').replace('三角', 'triangle'))(frequency, 0.1)
    else:
        t_long, wave_long = gen.noise(0.1, 0.5, name.replace('ノイズ', '').replace('ホワイト', 'white').replace('ピンク', 'pink'))
    
    fft_result = fft(wave_long)
    freqs = fftfreq(len(wave_long), 1/sample_rate)
    positive_mask = freqs >= 0
    
    magnitude_db = 20 * np.log10(np.abs(fft_result[positive_mask]) + 1e-10)
    plt.plot(freqs[positive_mask], magnitude_db, linewidth=2)
    plt.title(f'{name}（スペクトル）')
    plt.xlabel('周波数 (Hz)')
    plt.ylabel('振幅 (dB)')
    plt.grid(True)
    plt.xlim(0, 3000)

plt.tight_layout()
plt.show()

print("基本波形の特徴:")
print("- サイン波: 純音、倍音なし")
print("- 方形波: 奇数倍音のみ")
print("- のこぎり波: 全倍音、1/n減衰")
print("- 三角波: 奇数倍音、1/n²減衰")
print("- ホワイトノイズ: 全周波数均等")
print("- ピンクノイズ: 1/f特性")

## 2. エンベロープジェネレーター

In [ ]:
class EnvelopeGenerator:
    def __init__(self, sample_rate=44100):
        self.sample_rate = sample_rate
        
    def adsr(self, attack_time, decay_time, sustain_level, release_time, 
            total_duration, note_duration=None):
        """ADSR エンベロープ"""
        if note_duration is None:
            note_duration = total_duration
            
        total_samples = int(total_duration * self.sample_rate)
        note_samples = int(note_duration * self.sample_rate)
        
        attack_samples = int(attack_time * self.sample_rate)
        decay_samples = int(decay_time * self.sample_rate)
        release_samples = int(release_time * self.sample_rate)
        
        # サスティン期間
        sustain_samples = note_samples - attack_samples - decay_samples
        
        envelope = np.zeros(total_samples)
        idx = 0
        
        # Attack
        if attack_samples > 0:
            envelope[idx:idx+attack_samples] = np.linspace(0, 1, attack_samples)
            idx += attack_samples
            
        # Decay
        if decay_samples > 0 and idx < total_samples:
            end_idx = min(idx + decay_samples, total_samples)
            envelope[idx:end_idx] = np.linspace(1, sustain_level, end_idx - idx)
            idx = end_idx
            
        # Sustain
        if sustain_samples > 0 and idx < total_samples:
            end_idx = min(idx + sustain_samples, total_samples)
            envelope[idx:end_idx] = sustain_level
            idx = end_idx
            
        # Release
        if release_samples > 0 and idx < total_samples:
            end_idx = min(idx + release_samples, total_samples)
            envelope[idx:end_idx] = np.linspace(sustain_level, 0, end_idx - idx)
            
        return envelope
        
    def exponential_decay(self, duration, decay_rate=2.0):
        """指数減衰エンベロープ"""
        samples = int(duration * self.sample_rate)
        t = np.linspace(0, duration, samples)
        return np.exp(-decay_rate * t)
        
    def percussive(self, duration, attack_ratio=0.1):
        """打楽器風エンベロープ"""
        samples = int(duration * self.sample_rate)
        attack_samples = int(attack_ratio * samples)
        
        envelope = np.zeros(samples)
        
        # 急速なアタック
        envelope[:attack_samples] = np.linspace(0, 1, attack_samples)
        
        # 指数減衰
        decay_samples = samples - attack_samples
        if decay_samples > 0:
            t_decay = np.linspace(0, duration * (1 - attack_ratio), decay_samples)
            envelope[attack_samples:] = np.exp(-3 * t_decay / duration)
            
        return envelope

# エンベロープの比較
env_gen = EnvelopeGenerator(sample_rate)
duration = 2.0
t_env = np.linspace(0, duration, int(sample_rate * duration))

envelopes = {
    'ADSR (Piano風)': env_gen.adsr(0.01, 0.3, 0.3, 1.0, duration, 1.5),
    'ADSR (Organ風)': env_gen.adsr(0.1, 0.1, 0.8, 0.5, duration, 1.8),
    'ADSR (Pad風)': env_gen.adsr(0.5, 0.3, 0.7, 0.8, duration, 1.5),
    '指数減衰': env_gen.exponential_decay(duration, 2.0),
    '打楽器風': env_gen.percussive(duration, 0.05)
}

# エンベロープを音に適用
t_note, base_note = gen.sawtooth_wave(220, duration)  # A3のこぎり波

plt.figure(figsize=(16, 10))

for i, (env_name, envelope) in enumerate(envelopes.items()):
    # エンベロープ形状
    plt.subplot(3, 5, i + 1)
    plt.plot(t_env, envelope, linewidth=2)
    plt.title(f'{env_name}（エンベロープ）')
    plt.ylabel('振幅')
    plt.grid(True)
    plt.ylim(0, 1.1)
    
    # エンベロープ適用後の波形
    plt.subplot(3, 5, i + 6)
    modulated_note = base_note * envelope
    plt.plot(t_note[:2000], modulated_note[:2000], linewidth=1)  # 最初の部分のみ
    plt.title(f'{env_name}（適用後波形）')
    plt.ylabel('振幅')
    plt.grid(True)
    
    # スペクトログラム
    plt.subplot(3, 5, i + 11)
    f, t_spec, Sxx = signal.spectrogram(modulated_note, sample_rate, 
                                       nperseg=256, noverlap=128)
    plt.pcolormesh(t_spec, f, 10 * np.log10(Sxx + 1e-10), 
                  shading='gouraud', cmap='viridis')
    plt.title(f'{env_name}（スペクトログラム）')
    plt.ylabel('周波数 (Hz)')
    plt.ylim(0, 1500)
    
    if i == 4:  # 最後の列
        plt.xlabel('時間 (秒)')
        plt.subplot(3, 5, i + 6)
        plt.xlabel('時間 (秒)')
        plt.subplot(3, 5, i + 11)
        plt.xlabel('時間 (秒)')

plt.tight_layout()
plt.show()

print("エンベロープの種類と特徴:")
print("- Piano風: 速いアタック、中程度のディケイとリリース")
print("- Organ風: 中程度のアタック、長いサスティン")
print("- Pad風: 遅いアタック、長いサスティンとリリース")
print("- 指数減衰: 楽器の自然な減衰をシミュレート")
print("- 打楽器風: 非常に速いアタック、急速な減衰")

## 3. 簡易シンセサイザー

In [ ]:
class SimpleSynthesizer:
    def __init__(self, sample_rate=44100):
        self.sample_rate = sample_rate
        self.wave_gen = WaveformGenerator(sample_rate)
        self.env_gen = EnvelopeGenerator(sample_rate)
        
    def generate_note(self, frequency, duration, waveform='sine', 
                     attack=0.1, decay=0.1, sustain=0.7, release=0.2,
                     amplitude=1.0):
        """シンセサイザーの音符を生成"""
        
        # 波形生成
        if waveform == 'sine':
            t, wave = self.wave_gen.sine_wave(frequency, duration, amplitude)
        elif waveform == 'square':
            t, wave = self.wave_gen.square_wave(frequency, duration, amplitude)
        elif waveform == 'sawtooth':
            t, wave = self.wave_gen.sawtooth_wave(frequency, duration, amplitude)
        elif waveform == 'triangle':
            t, wave = self.wave_gen.triangle_wave(frequency, duration, amplitude)
        else:
            t, wave = self.wave_gen.sine_wave(frequency, duration, amplitude)
            
        # エンベロープ適用
        envelope = self.env_gen.adsr(attack, decay, sustain, release, duration)
        
        return t, wave * envelope
        
    def generate_chord(self, frequencies, duration, waveform='sine', **envelope_params):
        """和音の生成"""
        chord_signal = None
        
        for freq in frequencies:
            t, note = self.generate_note(freq, duration, waveform, **envelope_params)
            
            if chord_signal is None:
                chord_signal = note
            else:
                chord_signal += note
                
        # 正規化
        chord_signal = chord_signal / len(frequencies)
        
        return t, chord_signal
        
    def generate_sequence(self, note_sequence, note_duration=0.5, waveform='sine', **envelope_params):
        """音符シーケンスの生成"""
        sequence_signal = np.array([])
        
        for freq in note_sequence:
            if freq > 0:  # 0は休符
                t, note = self.generate_note(freq, note_duration, waveform, **envelope_params)
                sequence_signal = np.concatenate([sequence_signal, note])
            else:
                # 休符（無音）
                silence = np.zeros(int(note_duration * self.sample_rate))
                sequence_signal = np.concatenate([sequence_signal, silence])
                
        total_time = len(sequence_signal) / self.sample_rate
        t_sequence = np.linspace(0, total_time, len(sequence_signal))
        
        return t_sequence, sequence_signal

# シンセサイザーのデモ
synth = SimpleSynthesizer(sample_rate)

# 1. 異なる波形での同じ音符
frequency = 440  # A4
duration = 1.5

waveforms = ['sine', 'square', 'sawtooth', 'triangle']
waveform_signals = {}

for waveform in waveforms:
    t, note = synth.generate_note(frequency, duration, waveform, 
                                 attack=0.05, decay=0.2, sustain=0.5, release=0.4)
    waveform_signals[waveform] = (t, note)

# 2. 和音の生成（Cメジャー）
c_major_freqs = [261.63, 329.63, 392.00]  # C, E, G
t_chord, chord = synth.generate_chord(c_major_freqs, 2.0, 'sawtooth',
                                     attack=0.1, decay=0.3, sustain=0.6, release=0.5)

# 3. メロディーシーケンス（「きらきら星」の一部）
twinkle_freqs = [261.63, 261.63, 392.00, 392.00, 440.00, 440.00, 392.00, 0,  # ド ド ソ ソ ラ ラ ソ 休
                349.23, 349.23, 329.63, 329.63, 293.66, 293.66, 261.63]    # ファ ファ ミ ミ レ レ ド

t_melody, melody = synth.generate_sequence(twinkle_freqs, 0.4, 'triangle',
                                         attack=0.02, decay=0.1, sustain=0.7, release=0.15)

# 可視化
plt.figure(figsize=(18, 15))

# 波形比較
for i, (waveform, (t, signal)) in enumerate(waveform_signals.items()):
    # 時間波形
    plt.subplot(4, 4, i*2 + 1)
    plt.plot(t, signal, linewidth=1)
    plt.title(f'{waveform.capitalize()}波（時間）')
    plt.ylabel('振幅')
    plt.grid(True)
    
    # スペクトログラム
    plt.subplot(4, 4, i*2 + 2)
    f, t_spec, Sxx = signal.spectrogram(signal, sample_rate, nperseg=256, noverlap=128)
    plt.pcolormesh(t_spec, f, 10 * np.log10(Sxx + 1e-10), 
                  shading='gouraud', cmap='viridis')
    plt.title(f'{waveform.capitalize()}波（スペクトログラム）')
    plt.ylabel('周波数 (Hz)')
    plt.ylim(0, 2000)
    
    if i == 3:
        plt.xlabel('時間 (秒)')
        plt.subplot(4, 4, i*2 + 2)
        plt.xlabel('時間 (秒)')

# 和音の可視化
plt.subplot(4, 4, 9)
plt.plot(t_chord, chord, 'g-', linewidth=1)
plt.title('Cメジャーコード（時間）')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True)

plt.subplot(4, 4, 10)
f_chord, t_chord_spec, Sxx_chord = signal.spectrogram(chord, sample_rate, nperseg=256, noverlap=128)
plt.pcolormesh(t_chord_spec, f_chord, 10 * np.log10(Sxx_chord + 1e-10),
              shading='gouraud', cmap='plasma')
plt.title('Cメジャーコード（スペクトログラム）')
plt.xlabel('時間 (秒)')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 1000)

# メロディーの可視化
plt.subplot(4, 4, 11)
plt.plot(t_melody, melody, 'm-', linewidth=1)
plt.title('きらきら星メロディー（時間）')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True)

plt.subplot(4, 4, 12)
f_melody, t_melody_spec, Sxx_melody = signal.spectrogram(melody, sample_rate, nperseg=256, noverlap=128)
plt.pcolormesh(t_melody_spec, f_melody, 10 * np.log10(Sxx_melody + 1e-10),
              shading='gouraud', cmap='hot')
plt.title('きらきら星メロディー（スペクトログラム）')
plt.xlabel('時間 (秒)')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 1000)

# 総合スペクトル比較
plt.subplot(4, 4, 13)
for waveform, (t, signal) in waveform_signals.items():
    # 中間部分のスペクトルを取得
    mid_start = len(signal) // 3
    mid_end = 2 * len(signal) // 3
    segment = signal[mid_start:mid_end]
    
    fft_result = fft(segment * np.hanning(len(segment)))
    freqs = fftfreq(len(segment), 1/sample_rate)
    positive_mask = freqs >= 0
    
    magnitude_db = 20 * np.log10(np.abs(fft_result[positive_mask]) + 1e-10)
    plt.plot(freqs[positive_mask], magnitude_db, label=waveform.capitalize(), linewidth=2)

plt.title('波形別スペクトル比較')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅 (dB)')
plt.legend()
plt.grid(True)
plt.xlim(0, 2500)

# エンベロープの効果比較
plt.subplot(4, 4, 14)
envelope_types = [
    ('速い', {'attack': 0.01, 'decay': 0.1, 'sustain': 0.3, 'release': 0.2}),
    ('標準', {'attack': 0.1, 'decay': 0.2, 'sustain': 0.6, 'release': 0.3}),
    ('遅い', {'attack': 0.3, 'decay': 0.4, 'sustain': 0.8, 'release': 0.6})
]

for env_name, env_params in envelope_types:
    envelope = synth.env_gen.adsr(env_params['attack'], env_params['decay'], 
                                 env_params['sustain'], env_params['release'], 1.0)
    t_env = np.linspace(0, 1.0, len(envelope))
    plt.plot(t_env, envelope, label=f'{env_name}ADSR', linewidth=2)

plt.title('エンベロープ比較')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print("シンセサイザーの構成要素:")
print("1. オシレーター: 基本波形の生成")
print("2. エンベロープジェネレーター: 時間変化の制御")
print("3. 和音生成: 複数周波数の合成")
print("4. シーケンス生成: メロディーの自動生成")
print("5. 各パラメータの調整により多様な音色を作成可能")

## 練習問題

1. LFO（低周波オシレーター）を使ったビブラートやトレモロ効果を実装してみましょう
2. フィルターを組み込んだより高機能なシンセサイザーを作成してみましょう
3. FM合成やAM合成を実装して複雑な音色を作ってみましょう